<a href="https://colab.research.google.com/github/Dhamu785/Assignments/blob/main/Intermediate/02_Intermediate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installing and Importing libraries**

In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
import pandas as pd
import numpy as np

## **Downloading BERT models**

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

## **Loading data**

In [9]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks YouTube tutorials/zzz Assignments/02_intermediate/Train.csv')
df.head(3)

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0


In [24]:
df.memory_usage()

Index       128
text     320000
label    320000
dtype: int64

In [25]:
len(df)

40000

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


## **Encoding and decoding the data**

In [37]:
df['text'].iloc[0]

'I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.'

In [38]:
# Encoder
encoder = tokenizer.encode(df['text'].iloc[0], return_tensors='pt')
encoder

tensor([[  101,   151, 23650, 10700,   113,   144,   119, 10950,   114, 58997,
         10110, 51939, 10103, 31782, 72380,   119, 10367, 11153, 23150, 10107,
         10160, 10666, 84447,   119, 11312, 11757,   107, 31782, 72380,   107,
         11364, 10666,   117, 10770, 75612, 10110, 10515, 10666,   119, 11312,
         10367, 20043, 10114, 10346, 77805, 10362, 13064,   119, 10181, 10399,
         20043, 10114, 10346, 13143,   119, 95009, 12090, 10195,   126, 11365,
         10144, 10901, 11857,   119,   151, 12384, 11153, 12171, 10114, 11811,
         10103, 13113, 13365, 10285, 10578, 11008, 13168,   143, 11158, 56631,
         10111, 10108, 11523,   151, 46747, 10146,   143, 14763,   119, 12548,
         48921, 10563, 31021, 54894, 36592, 10422,   119, 10103, 10902, 11053,
         11291, 10140, 10103, 52539, 58990, 18245, 51301,   119, 10497, 10203,
         10197, 12296, 40952, 10171, 10103, 11383, 16843, 10108, 10103, 31782,
         72380,   119, 67715, 46943, 11607, 24989, 1

In [39]:
decoder = tokenizer.decode(encoder[0])
decoder

'[CLS] i grew up ( b. 1965 ) watching and loving the thunderbirds. all my mates at school watched. we played " thunderbirds " before school, during lunch and after school. we all wanted to be virgil or scott. no one wanted to be alan. counting down from 5 became an art form. i took my children to see the movie hoping they would get a glimpse of what i loved as a child. how bitterly disappointing. the only high point was the snappy theme tune. not that it could compare with the original score of the thunderbirds. thankfully early saturday mornings one television channel still plays reruns of the series gerry anderson and his wife created. jonatha frakes should hand in his directors chair, his version was completely hopeless. a waste of film. utter rubbish. a cgi remake may be acceptable but replacing marionettes with homo sapiens subsp. sapiens was a huge error of judgment. [SEP]'

## **Model predictions**
- Since it is a pre trained model there is no need of training.
- It is measured by a scale from 1 - 5.
- 1 means BAD
- 5 means BEST

In [61]:
sentiments = {1:'Bad',2:'Need to improve',3:'Neutral',4:'Good',5:'Best'}
def predict(review):
    encoded = tokenizer.encode(review, return_tensors='pt')
    result = model(encoded)
    int_res = int(torch.argmax(result.logits))+1
    return sentiments[int_res]

In [62]:
# PREDICTIONS
for i in range(150,160):
    print(f"Result = {predict(df['text'].iloc[i][:512])}")

Result = Bad
Result = Good
Result = Need to improve
Result = Best
Result = Need to improve
Result = Bad
Result = Good
Result = Good
Result = Good
Result = Best
